### Reference:

1 ) https://python.langchain.com/docs/integrations/llms/huggingface_hub

2 ) https://python.langchain.com/docs/integrations/chat/huggingface

### This notebook is to demostrate:

1 ) How to work with tools in langchain

2 ) How to create your own tools

3 ) How to use a HuggingFace Hub llm

4 ) How to create a ChatHaggingFace wrapper - chat model

5 ) How to create the chat agent

6 ) How to create an Agent Executer with your tools and chat model

7 ) How to set observation as a chat model stop condition

8 ) How to invoke an Agent Executor and get results in verbose mode - try different prompts and see if it works

Note: HuggingFaceHub and ChatHaggingFace are updated locally and are imported via llm_utils library

This notebook is created in Amazon SageMaker Studio Lab - free account

### Install required libraries

In [1]:
%pip install --upgrade --quiet python_dotenv langchain pip install langchainhub huggingface_hub  text-generation transformers sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from huggingface_hub import login
_ = load_dotenv(find_dotenv()) # read local .env file
hugging_face_access_token = os.environ['HUGGINGFACEHUB_API_TOKEN']
login(hugging_face_access_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/studio-lab-user/.cache/huggingface/token
Login successful


In [3]:
import llm_utils

In [4]:
llm = llm_utils.HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [5]:
# note you can override model paramaters like "max_new_token" when invoking the llm
llm.invoke(prompt="tell me a joke", max_new_tokens=12)

' about a fish\n\nthat will make me laugh out loud'

In [6]:
chat_model = llm_utils.ChatHuggingFace(llm=llm)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [7]:
chat_model.model_id

'HuggingFaceH4/zephyr-7b-beta'

In [8]:
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model._to_chat_prompt(messages)

"<|system|>\nYou're a helpful assistant</s>\n<|user|>\nWhat happens when an unstoppable force meets an immovable object?</s>\n<|assistant|>\n"

In [9]:
res = chat_model.invoke(messages)
print(res.content)

According to the popular philosophical concept, when an unstoppable force meets an immovable object, there is a paradox because both concepts seem to contradict each other. The idea of an unstoppable force implies that it can overcome any obstacle, while the concept of an immovable object suggests that it cannot be moved. This paradox raises questions about the nature of force and object, and whether such concepts are absolute or relative. Some interpretations suggest that the force may change or adapt in response to the object, while others suggest that the object may yield or give way under the force's pressure. However, the concept remains a thought-provoking paradox that challenges our understanding of physical laws and the limits of our knowledge.


In [10]:
from langchain_core.tools import tool
from langchain.tools.render import render_text_description

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

my_tools = [to_lower_case, multiply]
render_text_description(my_tools)

'lower_case: lower_case(input: str) -> str - Returns the input as all lower case.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(my_tools),
    tool_names=", ".join([t.name for t in my_tools]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [12]:
# create agent executor with tools
agent_executor = AgentExecutor(agent=agent, tools=my_tools, verbose=True)

In [20]:
agent_executor.invoke(
    {
        "input": "what is 5 multiplied by 6?"
    }
)



> Entering new AgentExecutor chain...
Here's the JSON blob to calculate the product of 5 and 6:

```json
{
  "action": "multiply",
  "action_input": {
    "first_int": 5,
    "second_int": 6
  }
}
```

Observation30Final Answer: The product of 5 and 6 is 30.

> Finished chain.


{'input': 'what is 5 multiplied by 6?',
 'output': 'The product of 5 and 6 is 30.'}